In [2]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from pathlib import Path

In [3]:
def train(df: pd.DataFrame) :
    # regression model
    rf = RandomForestRegressor(n_estimators=100, random_state=42)



    # TODO: remove date and add year, quarter, maybe wd_quarter (wd within quarter)

    categoric_col = ['ther_area', 'main_channel']

    pre_processor = ColumnTransformer(transformers=[('cat', OneHotEncoder(handle_unknown='ignore'), categoric_col)], remainder='passthrough')

    pipeline = Pipeline(steps=[('preprocessor', pre_processor), ('regressor', rf)])


    # missing monthly

    x_values = df[[ 'dayweek_sin','dayweek_cos', 'month_sin','month_cos', 'wd', 'ther_area','hospital_rate', 'main_channel', 'n_nwd_bef', 'n_nwd_aft']]

    y_values = df['phase']

    #x_train, x_test, y_train, y_test = train_test_split(x_values, y_values, test_size=0.0, random_state=42)

    print("\nstarted fitting\n")
    rf = pipeline.fit(x_values, y_values)
    print("\nfinished fitting\n")

    return rf



In [4]:
PATH = Path("../data")
#train_data = pd.read_parquet(PATH / "train_data.parquet")
train_data = pd.read_csv(PATH / "new_train_data.csv")

In [5]:
model = train(train_data)



started fitting



In [ ]:
#submission_data = pd.read_parquet(PATH / "submission_data.parquet")
submission_data = pd.read_csv(PATH / "new_submission_data.csv")

submission = pd.read_csv(PATH / "submission_template.csv")

# Fill in 'prediction' values of submission




In [ ]:

submission_data = submission_data[[['country','brand','phase','date','dayweek_sin','dayweek_cos', 'month_sin','month_cos', 'wd', 'ther_area','hospital_rate', 'main_channel', 'monthly', 'n_nwd_bef', 'n_nwd_aft']]]

submission['prediction'] = model.predict(submission_data)

# ...

# Save submission
SAVE_PATH = Path("../data")
submission.to_csv(SAVE_PATH / "submissionv2.csv", index=False)